In [1]:
import pandas as pd
import numpy as np
from supabase import create_client, Client
import joblib
from tensorflow.keras.models import load_model # type: ignore
from datetime import datetime

In [ ]:
import os
from dotenv import load_dotenv
from supabase import create_client

# Load environment variables
load_dotenv()

# Read Supabase credentials from .env
url = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_KEY")

# Create Supabase client
supabase = create_client(url, key)

In [3]:
def fetch_monthly_expense_features(user_id):
    response = (
        supabase
        .from_("transactions")
        .select("category_id, amount, created_at")
        .eq("user_id", user_id)
        .eq("transaction_type", "Expense")
        .order("created_at")
        .execute()
    )
    data = response.data if response.data else []
    if not data:
        raise Exception("No expense transactions found for this user")

    df = pd.DataFrame(data)
    df['created_at'] = pd.to_datetime(df['created_at']).dt.to_period('M').dt.to_timestamp()

    categories_resp = supabase.from_("categories").select("category_id, category_name").execute()
    categories_data = categories_resp.data if categories_resp.data else []
    if not categories_data:
        raise Exception("Failed to fetch categories")

    categories_df = pd.DataFrame(categories_data)
    df = df.merge(categories_df, on="category_id", how="left")
    df = df.dropna(subset=['category_name'])

    monthly_expenses = (
        df.groupby(['created_at', 'category_name'])['amount']
        .sum()
        .unstack(fill_value=0)
        .sort_index()
    )

    expected_features = ["Education", "Entertainment", "Fashion", "Food", "Lifestyle", "Transportation", "Health"]
    feature_matrix = pd.DataFrame(index=monthly_expenses.index)
    for feature in expected_features:
        feature_matrix[feature] = monthly_expenses.get(feature, 0)

    feature_matrix['Expenses'] = monthly_expenses.sum(axis=1)
    return feature_matrix.reset_index()

In [4]:
def predict_next_month_expense(user_id):
    # Fetch monthly features
    feature_matrix = fetch_monthly_expense_features(user_id)
    
    if len(feature_matrix) < 3:
        raise ValueError("Not enough data: at least 3 months of expense features are required.")
    
    scaler = joblib.load('scaler.save')
    model = load_model('lstm.h5')
    
    feature_columns = ["Education", "Entertainment", "Fashion", "Food", "Lifestyle", "Transportation", "Health"]
    
    # Use last 3 months of features
    last_3_months = feature_matrix.tail(3)
    
    # Print past 3 months expenses
    print("Past 3 months Expenses:")
    for date, expense in zip(last_3_months['created_at'], last_3_months['Expenses']):
        print(f"{date.strftime('%Y-%m')}: {expense:.2f}")
    
    # Scale features (exclude 'Expenses' and date)
    scaled_features = scaler.transform(last_3_months[feature_columns])
    
    # Prepare input shape (1, 3, 7)
    model_input = np.expand_dims(scaled_features, axis=0)
    
    # Predict next month expense
    predicted_expense = model.predict(model_input)[0, 0]
    
    # Calculate next month date (last date + 1 month)
    last_date = last_3_months['created_at'].max()
    next_month_date = (last_date + pd.offsets.MonthBegin(1))
    
    print(f"\nPredicted Expense for next month ({next_month_date.strftime('%Y-%m')}): {predicted_expense:.2f}")
    
    return predicted_expense

# Example usage:
user_id = "03afbd23-ccd9-46d9-b69a-56b7b1d9a6f0"
try:
    predict_next_month_expense(user_id)
except Exception as e:
    print("Error:", e)


C:\Users\menna\AppData\Local\Temp\ipykernel_2780\81130755.py:16: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['created_at'] = pd.to_datetime(df['created_at']).dt.to_period('M').dt.to_timestamp()



Past 3 months Expenses:
2025-04: 2811.40
2025-05: 2321.22
2025-06: 912.82
1/1 [==============================] - 1s 1s/step

Predicted Expense for next month (2025-07): 1709.76


In [5]:
def predict_next_month_expense(user_id):
    # Fetch monthly features
    feature_matrix = fetch_monthly_expense_features(user_id)
    
    # Remove current ongoing month if today is not the last day of the month
    today = pd.Timestamp(datetime.today().date())
    last_date = feature_matrix['created_at'].max()
    
    # Check if last_date month is current month and partial
    if (last_date.year == today.year) and (last_date.month == today.month):
        # Drop last_date month from data (partial month)
        feature_matrix = feature_matrix[feature_matrix['created_at'] < last_date]
    
    if len(feature_matrix) < 3:
        raise ValueError("Not enough full months of data (need at least 3).")
    
    scaler = joblib.load('scaler.save')
    model = load_model('lstm.h5')
    
    feature_columns = ["Education", "Entertainment", "Fashion", "Food", "Lifestyle", "Transportation", "Health"]
    
    # Use last 3 full months of features
    last_3_months = feature_matrix.tail(3)
    
    # Print past 3 months expenses
    print("Past 3 full months Expenses:")
    for date, expense in zip(last_3_months['created_at'], last_3_months['Expenses']):
        print(f"{date.strftime('%Y-%m')}: {expense:.2f}")
    
    # Scale features (exclude 'Expenses' and date)
    scaled_features = scaler.transform(last_3_months[feature_columns])
    
    # Prepare input shape (1, 3, 7)
    model_input = np.expand_dims(scaled_features, axis=0)
    
    # Predict next month expense
    predicted_expense = model.predict(model_input)[0, 0]
    
    # Calculate next month date (last date + 1 month)
    next_month_date = (last_3_months['created_at'].max() + pd.offsets.MonthBegin(1))
    
    print(f"\nPredicted Expense for next month ({next_month_date.strftime('%Y-%m')}): {predicted_expense:.2f}")
    
    return predicted_expense

# Example usage:
user_id = "03afbd23-ccd9-46d9-b69a-56b7b1d9a6f0"
try:
    predict_next_month_expense(user_id)
except Exception as e:
    print("Error:", e)

C:\Users\menna\AppData\Local\Temp\ipykernel_2780\81130755.py:16: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['created_at'] = pd.to_datetime(df['created_at']).dt.to_period('M').dt.to_timestamp()


Past 3 full months Expenses:
2025-03: 2937.38
2025-04: 2811.40
2025-05: 2321.22
1/1 [==============================] - 1s 939ms/step

Predicted Expense for next month (2025-06): 1709.68
